# YOLO Object Detection on Live Camera Feed (Raspberry Pi Compatible)
This notebook demonstrates how to run YOLO object detection on a live camera feed, with steps and code suitable for running on a Raspberry Pi.

## 1. Install Required Libraries
Install the necessary libraries for YOLO object detection and camera access. These commands are suitable for Raspberry Pi OS.

In [ ]:
# Install required libraries
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip install opencv-python
!pip install yolov5

# For Raspberry Pi camera support, you may need to install additional packages:
# !sudo apt-get update && sudo apt-get install python3-picamera
# If using PiCamera, uncomment the above line and use 'picamera' library in code.

## 2. Import Libraries
Import all necessary Python libraries for YOLO object detection and camera access.

In [ ]:
# Import required libraries
import cv2
import torch
import numpy as np
import time
# If using PiCamera, import the following:
# from picamera.array import PiRGBArray
# from picamera import PiCamera

## 3. Set Up Camera for Live Feed
Initialize the camera using OpenCV. For Raspberry Pi, you may use the default camera index (0) or PiCamera for better performance.

In [ ]:
# Set up camera for live feed
# For 3rd party USB camera, try index 1 (change if needed)
camera = cv2.VideoCapture(1)

# For Raspberry Pi Camera Module, use PiCamera (uncomment below if needed)
# camera = PiCamera()
# camera.resolution = (640, 480)
# camera.framerate = 32
# rawCapture = PiRGBArray(camera, size=(640, 480))

# Test if camera opened successfully
if not camera.isOpened():
    print("Error: Could not open camera. Try changing the index (e.g., 0, 1, 2) to match your USB camera.")
else:
    print("Camera initialized successfully (USB camera, index 1).")

## 4. Load YOLO Model
Load a pre-trained YOLO model (e.g., YOLOv5) using PyTorch. This example uses the ultralytics/yolov5 repository.

In [ ]:
# Load YOLOv5 model
# This will download the model if not already present
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.eval()
print("YOLOv5 model loaded.")

## 5. Process Live Camera Feed and Detect Objects
Capture frames from the camera, preprocess them, and run YOLO detection in a loop.

In [ ]:
# Process live camera feed and detect objects
try:
    while True:
        ret, frame = camera.read()
        if not ret:
            print("Failed to grab frame")
            break
        # YOLO expects RGB images
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = model(img)
        # results.xyxy[0] contains [x1, y1, x2, y2, confidence, class]
        # We'll draw these in the next step
        # Show frame with detections (see next cell)
        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except KeyboardInterrupt:
    print("Interrupted by user.")

## 6. Display Detection Results in Real-Time
Draw bounding boxes and labels on detected objects, and display the annotated video stream.

In [ ]:
# Display detection results in real-time
try:
    while True:
        ret, frame = camera.read()
        if not ret:
            print("Failed to grab frame")
            break
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = model(img)
        # Draw bounding boxes and labels
        for *box, conf, cls in results.xyxy[0]:
            x1, y1, x2, y2 = map(int, box)
            label = f"{model.names[int(cls)]} {conf:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(frame, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
        cv2.imshow('YOLO Live Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except KeyboardInterrupt:
    print("Interrupted by user.")
finally:
    camera.release()
    cv2.destroyAllWindows()